# Testing out trained model for AffectNet

http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/

## Importing libraries

In [1]:
import tflearn
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd
import os, sys
from PIL import Image

When we want to restore a trained model, we not only have to restore the graph and weights, but also prepare a new feed_dict that will feed the new training data to the network. If we just want to run the same network with different data, you can simply pass the new data via feed_dict to the network.

## Defining the model

In [3]:
tf.reset_default_graph()

In [4]:
n=5
# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
img_prep.add_featurewise_zero_center(per_channel=True, mean=[ 0.38888048 , 0.43694749,  0.53946589])

# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_crop([49, 49], padding=4)

# Building Residual Network
net = tflearn.input_data(shape=[None, 49, 49, 3],
                         data_preprocessing=img_prep,data_augmentation=img_aug)
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.resnext_block(net, n, 16, 32)
net = tflearn.resnext_block(net, 1, 32, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 32, 32)
net = tflearn.resnext_block(net, 1, 64, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 64, 32)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)
# Regression
net = tflearn.fully_connected(net, 2)

regression = tflearn.regression(net, optimizer='sgd', loss='mean_square',
                                metric='R2', learning_rate=0.01)
# Training
model = tflearn.DNN(regression,
  tensorboard_dir='Logs/',
  checkpoint_path='Snapshots/model_resnet_',
  max_checkpoints=10,
  clip_gradients=0.)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [5]:
model.load('model_resnet_-332000',weights_only=True)

INFO:tensorflow:Restoring parameters from /media/amogh/Stuff/CMU/datasets/AffectNet/Trained Networks (TfLearn)/Valence_Arousal/meta-data/model_resnet_-332000


## Trying on manually annotated data, used for training

In [6]:
path_validation_csv = "/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_file_lists/validation.csv"
folder_annotated_validation_images = "/media/amogh/Stuff/CMU/datasets/AffectNet/Manually_Annotated_Images/"

In [7]:
df_val = pd.read_csv(path_validation_csv,header=None)
df_val.head()

,0,1,2,3,4,5,6,7,8
0,459/81456263be241927c7a59a2646f88c2700ce4b7cba...,147,147,981,981,233.58;512.99;246.97;623.96;262.53;733.87;285....,7,-0.653310,0.653310
1,680/cfa0c679da3dbe9f01e92cdeda2da1065aa50e7bf0...,99,99,666,666,98.98;341.86;107.43;422.6;123.41;503.32;144.97...,0,-0.176846,-0.077640
2,1042/83dac9d7eec7fb0bfef79b4d35ddc69a6f7fb458d...,16,16,133,133,42.69;79.9;42.94;90.59;43.83;101.27;44.2;112.5...,0,-0.367789,0.183895
3,887/49e4872adc501d5e69b8898930bbd4f4e037022874...,78,78,522,522,49.66;304.48;48.91;361.36;52.08;421.7;64.76;48...,4,-0.062911,0.532327
4,1057/0745dd0386476ae193ca709034378c9207e48ae88...,36,36,258,258,21.82;115.14;18.22;152.92;20.65;190.43;26.14;2...,0,0.004839,-0.004839


In [8]:
list_image_paths = df_val.iloc[:,0].values

In [9]:
df_va_validation = df_val.iloc[:,[0,len(df_val.columns)-2,len(df_val.columns)-1]]
df_va_validation.columns = ["image","valence", "arousal"]
print("shape is: ", df_va_validation.shape)
(df_va_validation.head())

shape is:  (5500, 3)


,image,valence,arousal
0,459/81456263be241927c7a59a2646f88c2700ce4b7cba...,-0.653310,0.653310
1,680/cfa0c679da3dbe9f01e92cdeda2da1065aa50e7bf0...,-0.176846,-0.077640
2,1042/83dac9d7eec7fb0bfef79b4d35ddc69a6f7fb458d...,-0.367789,0.183895
3,887/49e4872adc501d5e69b8898930bbd4f4e037022874...,-0.062911,0.532327
4,1057/0745dd0386476ae193ca709034378c9207e48ae88...,0.004839,-0.004839


## Predictions

In [11]:
new_df = pd.DataFrame(columns=["image","valence", "arousal","valence_pred", "arousal_pred"])

for im_path in list_image_paths:
    im_abs_path = folder_annotated_validation_images + im_path
    if os.path.exists(im_abs_path):
        img = Image.open(im_abs_path)
        img = img.resize((49,49))
        data = np.asarray(img,dtype='float32')/255.
        #predict called for the model
        res = model.predict(np.expand_dims(data,0))
        val_truth = (df_va_validation[df_va_validation['image'] == im_path]['valence'].values[0])
        ar_truth = (df_va_validation[df_va_validation['image'] == im_path]['arousal'].values[0])
        df_new_row = pd.DataFrame([[im_path,val_truth,ar_truth,res[0][0],res[0][1]]],columns=["image","valence", "arousal","valence_pred", "arousal_pred"])
        new_df=new_df.append(df_new_row)
new_df

,image,valence,arousal,valence_pred,arousal_pred
0,1042/83dac9d7eec7fb0bfef79b4d35ddc69a6f7fb458d...,-0.367789,0.183895,-0.240533,1.696213
0,1057/0745dd0386476ae193ca709034378c9207e48ae88...,0.004839,-0.004839,-0.151542,1.601383
0,1054/89e3c31e878a050478f6a3660f44ad11345e5aec4...,-0.539683,0.357143,-0.168639,1.640910
0,1036/fa7596e96d7cb04dee33b39bfd78d3df4a03c5213...,-0.182540,0.563492,-0.198758,1.657848
0,1011/10d3e8fe4d9296e496d0ca2819e99ef31330e7daa...,-0.643631,0.648471,-0.210834,1.644388
0,1014/12feec6cb6bb2e329a83dcf7cd4f87ff19eab7ad1...,-0.174603,0.547619,-0.237745,1.735962
0,1013/6ae6c8329c31a2405a4b94d05429bc4c2dfe907ab...,-0.595238,0.658730,-0.193775,1.685243
0,1024/3cb72e71aafc9364661f349d16342f06a19167f55...,0.125086,0.487405,-0.191265,1.649030
0,1045/c4f8afc26d293aaebe320d74afe459a6106f19cc8...,-0.039682,-0.031746,-0.200019,1.657869
0,1008/29a06c26e44edf8fda7a49969280c89ff0b6bc98d...,0.825397,0.166667,-0.234920,1.696337


In [ ]:
new_df = pd.DataFrame(columns=["image","valence", "arousal","valence_pred", "arousal_pred"])

for im_path in list_image_paths:
    im_abs_path = folder_annotated_validation_images + im_path
    if os.path.exists(im_abs_path):
        img = Image.open(im_abs_path)
        img = img.resize((49,49))
        data = np.asarray(img,dtype='float32')
        res = model.predict(np.expand_dims(data-mean,0))
        val_truth = (df_va_validation[df_va_validation['image']==im_path]['valence'].values[0])
        ar_truth = (df_va_validation[df_va_validation['image']==im_path]['arousal'].values[0])
        df_new_row = pd.DataFrame([[im_path,val_truth,ar_truth,res[0][0],res[0][1]]],columns=["image","valence", "arousal","valence_pred", "arousal_pred"])
        new_df=new_df.append(df_new_row)
new_df